In [23]:
import os
import csv
import base64
import requests
import random
from difflib import SequenceMatcher
from PIL import Image
from io import BytesIO

In [24]:
# Konfigurasi
LMSTUDIO_API_URL = "http://127.0.0.1:1234/v1/chat/completions"
SELECTED_MODEL_NAME = "google/gemma-3-4b"
DATASET_DIR = "C:\\Users\\User\\OneDrive\\Documents\\uas wahyu\\test"
LABEL_CSV = "label.csv"
OUTPUT_CSV = "results/ocr_predictions.csv"

In [25]:
# Variasi Prompt (anti-copy strategy)
PROMPT_VARIANTS = [
    "Tolong baca dan sebutkan nomor polisi kendaraan dari gambar ini. Jawaban hanya berupa isi nomor plat tanpa tambahan kata lainnya.",
    "Silakan ekstrak teks nomor plat kendaraan dari gambar ini, dan hanya tuliskan nomor plat-nya.",
    "Apa isi dari nomor plat kendaraan yang tampak pada gambar? Jawaban harus langsung berupa nomor plat tersebut.",
    "Identifikasi teks pada plat kendaraan di gambar ini. Harap jawab dengan format nomor plat saja.",
    "Baca plat nomor kendaraan dalam gambar ini, lalu berikan hanya nomor plat-nya tanpa kata tambahan."
]

In [26]:
# Encode Gambar
def encode_image(image_path):
    img = Image.open(image_path)
    img = img.convert("RGB")
    img = img.resize((64, 64))  # Resize agar lebih ringan
    buffered = BytesIO()
    img.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [27]:
# Hitung CER
def calculate_cer(pred, truth):
    matcher = SequenceMatcher(None, truth, pred)
    edits = 0
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag != 'equal':
            edits += max(i2 - i1, j2 - j1)
    return round(edits / max(len(truth), 1), 4)

In [28]:
# Load Label CSV
def load_ground_truth_from_csv(csv_path):
    gt_map = {}
    try:
        with open(csv_path, mode='r', encoding='utf-8') as file:
            reader = csv.reader(file)
            for row in reader:
                if len(row) >= 2 and "image" not in row[0].lower():
                    filename = os.path.basename(row[0]).strip()
                    label = row[1].strip().replace(" ", "").replace("-", "").upper()
                    gt_map[filename] = label
    except Exception as e:
        print(f"❌ Failed to read CSV: {e}")
    return gt_map

In [29]:
# Kirim Prompt dan Gambar ke LMStudio
def query_model(image_b64):
    prompt = random.choice(PROMPT_VARIANTS)  # Ambil prompt random

    payload = {
        "model": SELECTED_MODEL_NAME,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_b64}"}}
                ]
            }
        ],
        "temperature": 0.2,
        "stream": False
    }

    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(LMSTUDIO_API_URL, json=payload, headers=headers, timeout=60)
        if response.status_code == 200:
            result = response.json()
            return result["choices"][0]["message"]["content"].strip()
        else:
            print(f"API Error: {response.status_code} - {response.text}")
            return "ERROR"
    except Exception as e:
        print(f"❌ Request failed: {e}")
        return "ERROR"

In [30]:
# Fungsi Utama
def main():
    if not os.path.exists(DATASET_DIR):
        print(f"❌ Dataset folder tidak ditemukan: {DATASET_DIR}")
        return

    os.makedirs("results", exist_ok=True)
    labels = load_ground_truth_from_csv(LABEL_CSV)

    with open(OUTPUT_CSV, "w", newline='', encoding='utf-8') as out_csv:
        writer = csv.writer(out_csv)
        writer.writerow(["image", "ground_truth", "prediction", "CER_score"])

        total, errors, scores = 0, 0, []

        for fname in sorted(os.listdir(DATASET_DIR)):
            if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
                continue
            if fname not in labels:
                print(f"⚠️ No label for {fname}, skipped.")
                continue

            path = os.path.join(DATASET_DIR, fname)
            gt = labels[fname]
            print(f"📸 {fname}...")

            try:
                image_b64 = encode_image(path)
                pred = query_model(image_b64)
                pred = pred.replace(" ", "").replace("-", "").upper()
                cer = calculate_cer(pred, gt)
                writer.writerow([fname, gt, pred, cer])
                print(f"✅ GT: {gt} | Pred: {pred} | CER: {cer}")
                scores.append(cer)
                total += 1
            except Exception as e:
                print(f"❌ Error on {fname}: {e}")
                writer.writerow([fname, gt, "ERROR", 1.0])
                errors += 1

        if scores:
            avg = sum(scores) / len(scores)
            print(f"\n📈 Average CER: {avg:.4f}")
        print(f"\n🎉 Done! Processed: {total}, Errors: {errors}")


if __name__ == "__main__":
    main()

📸 test001_1.jpg...
✅ GT: B9140BCD | Pred: S140BCD | CER: 0.25
📸 test001_2.jpg...
✅ GT: B2407UZO | Pred: B2407UZO | CER: 0.0
📸 test001_3.jpg...
✅ GT: B2642PKM | Pred: B2842PKM | CER: 0.125
📸 test002_1.jpg...
✅ GT: BG1352AE | Pred: 1352AE | CER: 0.25
📸 test003_1.jpg...
✅ GT: B2634UZF | Pred: B2634UZF | CER: 0.0
📸 test003_2.jpg...
✅ GT: B1995JVK | Pred: B1995JVK | CER: 0.0
📸 test004_1.jpg...
✅ GT: B9062VEH | Pred: XYZ1234 | CER: 0.875
📸 test005_1.jpg...
✅ GT: DD3798KM | Pred: DD798 | CER: 0.375
📸 test006_1.jpg...
✅ GT: T1329KC | Pred: 1329KC | CER: 0.1429
📸 test007_1.jpg...
✅ GT: AD8865EE | Pred: AD865EE | CER: 0.125
📸 test008_1.jpg...
✅ GT: DK1157AAB | Pred: DK1157AAB | CER: 0.0
📸 test008_2.jpg...
✅ GT: AA1997FE | Pred: 1997FE | CER: 0.25
📸 test009_1.jpg...
✅ GT: H8518NA | Pred: H0518NA | CER: 0.1429
📸 test009_2.jpg...
✅ GT: K1649GB | Pred: KC1649GB | CER: 0.1429
📸 test010_1.jpg...
✅ GT: B9416PCN | Pred: B9416PCN | CER: 0.0
📸 test010_2.jpg...
✅ GT: H1147UY | Pred: H147UY | CER: 0.1429
📸 